The whole notebook takes around **10 minutes** to complete on a machine with Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz.

In [ ]:
import itertools

import numpy as np
import scipy.stats as ss
import statsmodels.stats.multitest as mt
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm_notebook as tqdm # progress bars

In [ ]:
# TCP = trigger coincidence process

def trigger_coincidences(X, E, taus, delta):
    # Compute the TCP K_{tr}^{delta,taus}(E, X) for time series 'X' and
    # event series 'E' using the sequence of thresholds 'taus' with a
    # time tolerance of 'delta'
    T = min(len(X), len(E))
    K_tr = np.zeros_like(taus, dtype=int)
    for i, tau in enumerate(taus):
        A = (X > tau)*1
        K_tr[i] = len([t for t in range(T-delta) if (E[t] == 1) and np.sum(A[t:t+delta+1]) >= 1])
    return K_tr

def _fit_gev_blockmaxima(X, blocksize):
    # Fit the parameters of the GEV distribution to the maxima of blocks
    # of size 'blocksize' in the time series 'X'
    T = len(X) - (len(X)%blocksize) # ignore remainder
    Mk = np.array([X[t:t+blocksize].max() for t in range(0, T, blocksize)])
    gev_params = ss.genextreme.fit(Mk)
    return gev_params

def tcp_params_fit(X, delta, taus):
    # Fit the parameters of the TCP Markov model (marginal and conditional probabilities)
    # for the time series 'X', with time tolerance 'delta' and threshold sequence 'taus'
    gev_params = _fit_gev_blockmaxima(X, delta+1)
    ps_marginal = np.array([1.-ss.genextreme.cdf(tau, *gev_params) for tau in taus])
    ps_conditional = np.ones_like(taus)*np.nan
    ps_conditional[1:] = np.array([ps_marginal[i]/ps_marginal[i-1] for i in range(1,len(taus))])
    return ps_marginal, ps_conditional

def tcp_marginal_expectation(N_E, tcp_params):
    # Compute the marginally expected TCP, i.e., the TCP that is obtained when taking the
    # pointwise expected values independently for each threshold
    return tcp_params[0]*N_E

def tcp_nll(K_tr, N_E, idx_start, tcp_params):
    # Compute the negative log-likelihood (test statistic value s) for the observed TCP 'K_tr',
    # when the event series has 'N_E' event occurrences. Use threshold at position 'idx_start'
    # as the first threshold (to shift attention to higher quantiles, default: 0), and
    # the TCP model parameters from 'tcp_params'
    ps_marginal, ps_conditional  = tcp_params
    return -(ss.binom.logpmf(K_tr[idx_start], N_E, ps_marginal[idx_start])
       + np.sum([ss.binom.logpmf(K_tr[i], K_tr[i-1], ps_conditional[i])
                     for i in range(idx_start+1, len(ps_marginal))]))

In [ ]:
def trigger_coincidences_pval(K_tr, N_E, pi):
    return ss.binom.pmf(K_tr, N_E, pi) + ss.binom.sf(K_tr, N_E, pi)

# Terrorist attacks

In [ ]:
date_range = pd.date_range("2015-01-01", "2017-12-31", freq="1D")

In [ ]:
# Download GTD from https://www.start.umd.edu/gtd/
# We use the 2014-2017 data set

#gtd = pd.read_csv("/path/to/gtd_14to17_0718dist.csv", low_memory=False)
gtd = pd.read_csv("/home/erik/Documents/data/globalterrorism-start/gtd_14to17_0718dist.csv", low_memory=False)
gtd = gtd.join(pd.to_datetime(gtd[["iyear", "imonth", "iday"]].rename({"iyear": "year", "imonth": "month", "iday": "day"}, axis="columns")).rename("date"))

In [ ]:
cols = ["date", "country_txt", "city", "nkill", "nwound", "gname"]

sel_islamist = ((gtd["gname"] == "Al-Qaida in the Arabian Peninsula (AQAP)")
                | (gtd["gname"] == "Islamic State of Iraq and the Levant (ISIL)")
                | (gtd["gname"] == "Jihadi-inspired extremists")
                | (gtd["gname"] == "Muslim extremists"))
sel_wena = ((gtd["region_txt"] == "Western Europe")
                | (gtd["region_txt"] == "North America"))
sel_mena = ((gtd["region_txt"] == "Middle East & North Africa"))
sel_min10wound = (gtd["nwound"] >= 10)

index = gtd.loc[
          sel_islamist
        & sel_wena
        & sel_min10wound
            ][cols].groupby("date").sum().index.copy()
eventseries = pd.Series(index=index, data=[1]*len(index), name="GTD").reindex(date_range).fillna(0)
eventseries.plot(figsize=(5,2))
plt.show()

E = eventseries.values

print("WENA-10: %d days with events. (Several entries at the same day are viewed as a single event.)" % eventseries.sum())
print("")
for i, r in enumerate(gtd[sel_wena & sel_islamist & sel_min10wound].itertuples()):
    print(i, r.date.date(), r.city, r.country_txt, r.nwound, r.nkill)

# Twitter volume

In [ ]:
timeseries_daily = pd.read_csv("twitter-volume.csv", index_col=0, parse_dates=[0])

In [ ]:
for c in timeseries_daily.columns:
    # preprocessing
    data = np.log2(timeseries_daily[c].copy()+1)
    data = data - data.rolling(window=30).mean()
    data = data.loc[date_range]

    plt.figure(figsize=(15,2))
    plt.title(c)
    for d in eventseries[eventseries == 1].index:
        plt.axvline(d, color="red", alpha=0.3)
    plt.plot(data)
    plt.xlim((date_range[0],date_range[-1]))
    plt.yticks([])
    plt.show()

# Analyses

In [ ]:
delta = 7

## Section 4.3. Examples.

#### Test whether Islamist terrorist attacks systematically trigger bursts of more than 1,000 posts per day on Twitter that contain the hashtag #stopislam.

In [ ]:
# NOTE: the raw time series used in this example is not stationary

c = "#stopislam"
data = timeseries_daily[c].loc[date_range].copy()
thresh = 1000

tcp_params = tcp_params_fit(data, delta, [thresh])
K_tr = trigger_coincidences(data.values, E, np.array([thresh]), delta)
pval = trigger_coincidences_pval(K_tr, E.sum(), tcp_params[0][0])

plt.figure(figsize=(15,2))
plt.title("%s: K=%d r=%.2f p=%.4f Delta=%d" % (c, K_tr, K_tr/E.sum(), pval, delta))
for d in data[data > thresh].index:
    plt.axvline(d, color="blue", alpha=0.1)
for d in eventseries[eventseries == 1].index:
    plt.axvline(d, color="red", alpha=0.5)
plt.plot(data)
plt.xlim((date_range[0],date_range[-1]))
plt.yticks([])
plt.ylim((0,thresh))
plt.show()

#### Test the hypothesis that Islamist terrorist attacks systematically trigger bursts of #notinmyname usage that exceed the volume of 95% of all days.

In [ ]:
# NOTE: the raw time series used in this example is not stationary

c = "#notinmyname"
data = timeseries_daily[c].loc[date_range].copy()
thresh = np.percentile(data, 95)

tcp_params = tcp_params_fit(data, delta, [thresh])
K_tr = trigger_coincidences(data.values, E, np.array([thresh]), delta)
pval = trigger_coincidences_pval(K_tr, E.sum(), tcp_params[0][0])

plt.figure(figsize=(15,2))
plt.title("%s: K=%d r=%.2f p=%.4f Delta=%d" % (c, K_tr, K_tr/E.sum(), pval, delta))
for d in data[data > thresh].index:
    plt.axvline(d, color="blue", alpha=0.1)
for d in eventseries[eventseries == 1].index:
    plt.axvline(d, color="red", alpha=0.5)
plt.plot(data)
plt.xlim((date_range[0],date_range[-1]))
plt.yticks([])
plt.ylim((0,thresh))
plt.show()

# Section 5

In [ ]:
rhos = np.linspace(0.75, 1, 32)
idx_start = 0
delta = 7
simuls = 10000
alpha = 0.05

In [ ]:
X = {c: None for c in timeseries_daily.columns}
taus = {c: None for c in timeseries_daily.columns}
tcp_params = {c: None for c in timeseries_daily.columns}
simul_nlls = {c: None for c in timeseries_daily.columns}
simul_seqs = {c: None for c in timeseries_daily.columns}

for c in timeseries_daily.columns:
    data = np.log2(timeseries_daily[c].copy()+1)
    data = data - data.rolling(window=30).mean()
    data = data.loc[date_range]

    X[c] = data.values
    taus[c] = np.percentile(X[c],rhos*100)
    tcp_params[c] = tcp_params_fit(X[c], delta, taus[c])

    # Monte carlo simulations for statistical significance
    simul_nlls[c] = np.zeros(simuls)
    simul_seqs[c] = [None] * simuls
    for s in tqdm(range(simuls), total=simuls, desc=c):
        simul_E = np.random.permutation(E)
        simul_seqs[c][s] = trigger_coincidences(X[c], simul_E, taus[c], delta)
        simul_nlls[c][s] = tcp_nll(simul_seqs[c][s], simul_E.sum(), idx_start, tcp_params[c])

In [ ]:
plt.figure(figsize=(10,3))
for i, c in enumerate(timeseries_daily.columns):
    K_tr = trigger_coincidences(X[c], E, taus[c], delta)
    nll = tcp_nll(K_tr, E.sum(), idx_start, tcp_params[c]) 
    pval = (np.sum(simul_nlls[c] >= nll)+1)/(simuls+1)

    plt.subplot(1,3,i+1)
    plt.plot(rhos, tcp_marginal_expectation(E.sum(), tcp_params[c])/E.sum())
    plt.vlines(rhos, np.zeros_like(rhos), ss.binom.ppf(1-alpha, E.sum(), tcp_params[c][0])/E.sum(), alpha=0.2)
    plt.plot(rhos, K_tr/E.sum())
    plt.xlim((rhos[idx_start], rhos[-1]))
    plt.ylim((0,1))
    plt.title("%s (p%c%.4f)" % (c,
                    ('='  if pval > 0.0001 else '<'),
                    (pval if pval > 0.0001 else 0.0001)))
plt.show()

#### Alternative: Multiple hypothesis testing approach with pointwise $p$-values

In [ ]:
for i, c in enumerate(timeseries_daily.columns):
    K_tr = trigger_coincidences(X[c], E, taus[c], delta)
    pvals = np.zeros(len(rhos))
    for m, _ in enumerate(rhos):
        pvals[m] = trigger_coincidences_pval(K_tr[m], E.sum(), tcp_params[c][0][m])
    print(c)
    for method in ["b", "s", "hs", "h"]:
        reject, pvals_adj, _, _ = mt.multipletests(pvals, alpha=alpha, method=method)
        print("   ", method, "\t", "reject" if reject.any() else "no reject", "(p=%.4f)" % pvals_adj.min())